In [1]:
import json
import re
from openai import OpenAI

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key="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"
)

In [2]:
def calculate(expression):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Erro: {e}"

# função add para teste
def wikipedia_search(query):
    return f"Resultado simulado da Wikipedia para: {query}"

def get_planet_mass(planet):
    masses = {
        "earth": 5.972e24,
        "mars": 6.39e23,
        "jupiter": 1.898e27,
        "saturn": 5.683e26,
        "uranus": 8.681e25,
        "neptune": 1.024e26,
        "mercury": 3.301e23,
        "venus": 4.867e24,
        "sun": 1.989e30
    }
    return str(masses.get(planet.lower(), "Massa desconhecida"))


In [3]:
# log criando manualmente pois não consegui acessar modelo.messages
def save_log_to_json(messages, filename="historico_conversa.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(messages, f, indent=4, ensure_ascii=False, default=str)
    print(f"Log salvo em {filename}")

#### lista de ferramentas disponíveis

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculates a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The math expression to evaluate, e.g., '2 + 2'"
                    }
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "wikipedia_search",
            "description": "Search for a term on Wikipedia",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_planet_mass",
            "description": "Get the mass of a planet in kg",
            "parameters": {
                "type": "object",
                "properties": {
                    "planet": {"type": "string", "description": "The name of the planet, e.g., 'Mars'"}
                },
                "required": ["planet"],
            },
        },
    }
]

available_functions = {
    "calculate": calculate,
    "wikipedia_search": wikipedia_search,
    "get_planet_mass": get_planet_mass,
}

#### definindo o modelo

In [5]:
# novo modelo, atualizado com base nos modelos atuais disponíveis no site do groq
def steve_magal(question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the available tools to answer questions correctly."},
        {"role": "user", "content": question}
    ]

    # criando log manual para checagem posterior
    save_log_to_json(messages)

    print(f"--- Pergunta: {question} ---")

    # criando instancia do cliente
    response = client.chat.completions.create(
        model="openai/gpt-oss-20b", # modelo encontrado no site oficial do GROQ
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        print("--> O modelo decidiu usar ferramentas!")

        messages.append(response_message)

        # add informação no log
        save_log_to_json(messages)

        # iterando sobre as chamadas
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            print(f"--> Executando: {function_name} com args {function_args}")

            if function_name == "calculate":
                function_response = function_to_call(expression=function_args.get("expression"))
            elif function_name == "wikipedia_search":
                function_response = function_to_call(query=function_args.get("query"))
            elif function_name == "get_planet_mass":
                function_response = function_to_call(planet=function_args.get("planet"))

            print(f"--> Resultado: {function_response}")

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

           # add informação no log
            save_log_to_json(messages)

        # segunda chamada do modelo, para verificação das respostas
        second_response = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=messages,
        )
        # add informação no log
        save_log_to_json(messages)
        return second_response.choices[0].message.content

    else:
        return response_message.content

In [7]:
res = steve_magal("What is the mass of earth times 5?")
print(f"\nRESPOSTA FINAL:\n{res}")

Log salvo em historico_conversa.json
--- Pergunta: What is the mass of earth times 5? ---
--> O modelo decidiu usar ferramentas!
Log salvo em historico_conversa.json
--> Executando: get_planet_mass com args {'planet': 'Earth'}
--> Resultado: 5.972e+24
Log salvo em historico_conversa.json
Log salvo em historico_conversa.json

RESPOSTA FINAL:
The mass of the Earth is approximately \(5.972 \times 10^{24}\,\text{kg}\).  
Multiplying that by 5 gives:

\[
5.972 \times 10^{24}\,\text{kg} \times 5 \;=\; 2.986 \times 10^{25}\,\text{kg}
\]

So, five times the Earth's mass is about **\(2.986 \times 10^{25}\,\text{kg}\)**.


#### lendo o histórico de "pensamento" do modelo

In [12]:
def ler_log(nome_arquivo="historico_conversa.json"):
    try:
        with open(nome_arquivo, "r", encoding="utf-8") as f:
            dados = json.load(f)
            print(json.dumps(dados, indent=4, ensure_ascii=False))
    except FileNotFoundError:
        print("Arquivo de log não encontrado. Rode o agente primeiro.")

ler_log()

[
    {
        "role": "system",
        "content": "You are a helpful assistant. Use tools step-by-step to answer."
    },
    {
        "role": "user",
        "content": "Add up the mass of the first four planets in the solar system, from closest to farthest from the sun?"
    },
    "ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='fc_10fd51e1-4e83-4c83-95a6-24a11852949f', function=Function(arguments='{\"planet\":\"Mercury\"}', name='get_planet_mass'), type='function')], reasoning='The user asks: \"Add up the mass of the first four planets in the solar system, from closest to farthest from the sun?\" That means we need the masses of Mercury, Venus, Earth, Mars, and sum them.\\n\\nWe should use the get_planet_mass function to retrieve masses. Then sum them up. Use function calls. We\\'ll do each planet. Probably we need to handle units? The masses are presumabl

### executando o agente em loop

In [9]:
# função steve_magal alterada para retirar a "memória inicial"
# e permitir a passagem de informação através das interações em loop

def novo_steve_magal(messages):

    save_log_to_json(messages)

    response = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        print("   [Pensamento]: O modelo decidiu usar ferramentas!")
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            print(f"   [Ação]: Executando {function_name} com args {function_args}")

            if function_name == "calculate":
                function_response = function_to_call(expression=function_args.get("expression"))
            elif function_name == "wikipedia_search":
                function_response = function_to_call(query=function_args.get("query"))
            elif function_name == "get_planet_mass":
                function_response = function_to_call(planet=function_args.get("planet"))

            print(f"   [Observação]: Resultado: {function_response}")

            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            })
            save_log_to_json(messages)

        return "CONTINUE"

    else:
        return response_message.content

In [11]:
def loop_de_raciocinio(question, max_steps=10):
    print(f"--- Iniciando tarefa complexa: {question} ---")

    # "memória"
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use tools step-by-step to answer."},
        {"role": "user", "content": question}
    ]

    step = 0
    while step < max_steps:
        step += 1
        print(f"\n>> Passo de Raciocínio {step}")

        # agora a função recebe a "memória" da interação passada
        resultado = novo_steve_magal(messages)

        if resultado == "CONTINUE":
            continue
        else:
            return resultado

    return "Limite de passos atingido sem resposta final."

pergunta = "Add up the mass of the first four planets in the solar system, from closest to farthest from the sun?"
resposta_final = loop_de_raciocinio(pergunta)

print(f"\nRESPOSTA FINAL DO AGENTE:\n{resposta_final}")

--- Iniciando tarefa complexa: Add up the mass of the first four planets in the solar system, from closest to farthest from the sun? ---

>> Passo de Raciocínio 1
Log salvo em historico_conversa.json
   [Pensamento]: O modelo decidiu usar ferramentas!
   [Ação]: Executando get_planet_mass com args {'planet': 'Mercury'}
   [Observação]: Resultado: 3.301e+23
Log salvo em historico_conversa.json

>> Passo de Raciocínio 2
Log salvo em historico_conversa.json
   [Pensamento]: O modelo decidiu usar ferramentas!
   [Ação]: Executando get_planet_mass com args {'planet': 'Venus'}
   [Observação]: Resultado: 4.867e+24
Log salvo em historico_conversa.json

>> Passo de Raciocínio 3
Log salvo em historico_conversa.json
   [Pensamento]: O modelo decidiu usar ferramentas!
   [Ação]: Executando get_planet_mass com args {'planet': 'Earth'}
   [Observação]: Resultado: 5.972e+24
Log salvo em historico_conversa.json

>> Passo de Raciocínio 4
Log salvo em historico_conversa.json
   [Pensamento]: O modelo d